In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame (Extract)
* Pandas is used in the extraction process to read the CSV into a DataFrame.

In [2]:
csv_file = "../Resources/customer_data.csv"
customer_data_df = pd.read_csv(csv_file)
customer_data_df.head()

,id,first_name,last_name,email,gender,car
0,1,Benetta,Cancott,bcancott0@studiopress.com,Female,Scion
1,2,Lilyan,Cherry,lcherry1@deliciousdays.com,Female,Chrysler
2,3,Ezekiel,Benasik,ebenasik2@wikia.com,Male,Mercedes-Benz
3,4,Kennedy,Atlay,katlay3@so-net.ne.jp,Male,Buick
4,5,Sanford,Salmen,ssalmen4@reuters.com,Male,Lincoln


### Create new data with select columns (Transform)
* Pandas is again used in the transform process to clean the data to take in the columns that are needed for the `customer_name table`.

In [3]:
new_customer_data_df = customer_data_df[['id', 'first_name', 'last_name']].copy()
new_customer_data_df.head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Store JSON data into a DataFrame (Extract)

In [4]:
json_file = "../Resources/customer_location.json"
customer_location_df = pd.read_json(json_file)
customer_location_df.head()

,address,id,latitude,longitude,us_state
0,043 Mockingbird Place,1,39.1682,-86.5186,Indiana
1,4 Prentice Point,2,41.0938,-85.0707,Indiana
2,46 Derek Junction,3,32.7673,-96.7776,Texas
3,11966 Old Shore Place,4,39.0350,-94.3567,Missouri
4,5 Evergreen Circle,5,40.7808,-73.9772,New York


### Clean DataFrame (Transform)

In [5]:
new_customer_location_df = customer_location_df[["id", "address", "us_state"]].copy()
new_customer_location_df.head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York


### Connect to local database
* The user will be postgres and the password will be the same one used to connect to pgAdmin 4 server.
* You may need to `pip install psycopg2` package to connect to pgAdmin 4.

In [6]:
!pip install psycopg2-binary

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
  Using cached https://files.pythonhosted.org/packages/5c/1c/6997288da181277a0c29bc39a5f9143ff20b8c99f2a7d059cfb55163e165/psycopg2-2.8.3.tar.gz
  Stored in directory: /home/mkolawole/.cache/pip/wheels/48/06/67/475967017d99b988421b87bf7ee5fad0dad789dc349561786b
Successfully built psycopg2
You are using pip version 19.0.3, however version 19.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


* The connection string consists of `user:password@host/db_name`. The user will be postgres and the password will be the same one used to connect to pgAdmin 4 server. 

* The host will be `localhost:5432`. This is then passed into the create_engine function, which starts with `postgresql://` followed by the connection string
* The postgresql:// specifies to SQLALchemy the type of the database connection. This can take other parameters such as mysql and sqlite.

In [14]:
rds_connection_string = "postgres:p0stgr3s@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables
* `engine.table_names()` will confirm a successful connection by returning the names of the tables in the pgAdmin 4 database.

In [15]:
engine.table_names()

['customer_name', 'customer_location']

### Use pandas to load csv converted DataFrame into database

In [16]:
new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

### Use pandas to load json converted DataFrame into database

In [17]:
new_customer_location_df.to_sql(name='customer_location', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [18]:
pd.read_sql_query('select * from customer_name', con=engine).head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Confirm data has been added by querying the customer_location table

In [11]:
pd.read_sql_query('select * from customer_location', con=engine).head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York
